In [1]:
import pandas as pd

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name='titanic-optimization-local')

In [4]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, "titanic")
df = dataset.to_pandas_dataframe()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [5]:
df = df.drop_duplicates()

In [13]:
train = pd.get_dummies(df, columns=["Sex","Embarked","Cabin"], drop_first=True)
train = train.fillna(0)
X_train = train.drop(["Survived","PassengerId","Name","Ticket"],axis=1).values
y_train = train["Survived"].values

In [7]:
from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(X_train)

In [14]:
X_train.shape

(891, 154)

In [15]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

model = Sequential()

model.add(Dense(18, activation='relu', input_dim=154, kernel_initializer='uniform'))
model.add(Dropout(0.50))
model.add(Dense(60, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 18)                2790      
_________________________________________________________________
dropout_2 (Dropout)          (None, 18)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 60)                1140      
_________________________________________________________________
dropout_3 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 61        
Total params: 3,991
Trainable params: 3,991
Non-trainable params: 0
_________________________________________________________________


In [16]:
#Creating an Stochastic Gradient Descent
sgd = SGD(lr = 0.01, momentum = 0.9)

# Compiling our model
model.compile(optimizer = sgd,  loss = 'binary_crossentropy',  metrics = ['accuracy'])

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 60, epochs = 30, verbose=1)

Epoch 1/30
15/15 [==============================] - 0s 917us/step - loss: 0.6899 - accuracy: 0.6004
Epoch 2/30
15/15 [==============================] - 0s 1ms/step - loss: 0.6700 - accuracy: 0.6543
Epoch 3/30
15/15 [==============================] - 0s 903us/step - loss: 0.6557 - accuracy: 0.6498
Epoch 4/30
15/15 [==============================] - 0s 3ms/step - loss: 0.6499 - accuracy: 0.6622
Epoch 5/30
15/15 [==============================] - 0s 1ms/step - loss: 0.6396 - accuracy: 0.6611
Epoch 6/30
15/15 [==============================] - 0s 992us/step - loss: 0.6359 - accuracy: 0.6633
Epoch 7/30
15/15 [==============================] - 0s 1ms/step - loss: 0.6370 - accuracy: 0.6577
Epoch 8/30
15/15 [==============================] - 0s 1ms/step - loss: 0.6415 - accuracy: 0.6465
Epoch 9/30
15/15 [==============================] - 0s 1ms/step - loss: 0.6371 - accuracy: 0.6476
Epoch 10/30
15/15 [==============================] - 0s 927us/step - loss: 0.6420 - accuracy: 0.6644
Epoch 11/30

In [17]:
scores = model.evaluate(X_train, y_train, batch_size=30)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

30/30 [==============================] - 0s 717us/step - loss: 0.6187 - accuracy: 0.6554
accuracy: 65.54%


In [12]:
X_train.shape[1]

154